In [ ]:
import tqdm
import os
import pickle
import re
import numpy as np

In [ ]:
with open("selected_sentences.pickle", "rb") as f:
    sentences = pickle.load(f)

In [ ]:
term_lists = [
    (["兩造",], ["雙方"]),
    (["上揭", "上開", "前揭", "前開", "首揭"], ["前述", "上述"]),
    (["云云",], ["等陳述", "等語", "等等"]),
    (["可考", "可佐", "可按", "可稽", "可證", "足按", "足徵", "足稽", "足憑", "足證"], ["可以佐證", "可供證明", "可以證明", "足以佐證", "足以證明"]),
    (["迭",], ["接連", "多次"]),
    (["拘束",], ["限制",]),
    (["失所附麗",], ["失所依附",]),
    (["尚非無稽", "尚非無憑", "尚非無據", "尚非虛妄", "尚非臆造"], ["應可採信", "應屬事實", "並非全無依據", "並不是完全沒有依據"]),
    (["所載",], ["所記載",]),
    (["考諸", "徵諸", "觀諸", "稽之"], ["參考", "依照", "依據"]),
    (["質言之",], ["簡言之",]),
    (["相歧",], ["矛盾",]),
    (["即非法所不許", "依法即無不合",], ["符合法律規定",]),
    (["礙難採認",], ["難以認定", "難以採信", "不可採"]),
    (["矧",], ["況且",]),
    (["翻異"], ["推翻"]),
    (["乃",], ["於是",]),
    (["自白不諱", "供認不諱", "坦承不諱"], ["坦白承認"]),
    (["似無可採", "似屬無憑", "即無可採", "即屬無據", "尚無可採", "尚難憑採", "要非可信", "要屬虛言", "容非可採"], ["難以採信", "不可採信", "尚不足採信", "尚不足採證"]),
    (["顯有",], ["顯然有", "顯然屬於", "顯然是"])
]

for legal_terms, usual_terms in term_lists:

    terms = legal_terms+usual_terms
    print("|".join(terms))
    save_path = os.path.join("ngram_surprisals", "|".join(terms))

    if os.path.isfile(save_path): continue
    term_types = ["legal"]*len(legal_terms)+["usual"]*len(usual_terms)


    regex_str = "|".join(terms)
    idx = 0
    contexts = {term: [] for term in terms}

    context_sentences = []
    term_list = []
    long_sentences = []

    for sentence1, sentence2 in zip(sentences[:-1], sentences[1:]):
        long_sentences+=[sentence1+" "+sentence2]

        matches = list(re.finditer(regex_str, sentence2))
        if matches:
            for match in matches:
                if len(sentence2[:match.start()]) >= n:
                    context = sentence2[:match.start()][-n:]
                else:
                    context = sentence2[:match.start()].replace(".", "\.")
                    context = (sentence1+"."+context)[-n-len(".")-context.count("\."):]
                
                context_sentences.append(context)
                term_list.append(match.group())
    
    term_surprisals = {term: [] for term in terms}

    for context, term in zip(context_sentences, term_list):
        regex_str = context.replace("(", "\(").replace(")", "\)").replace("?", "\?").replace("!", "\!").replace(",", "\,")
        matches = re.finditer(regex_str, " ".join(sentences))
        context_count = len(list(matches))
        if context_count==0: raise ValueError

        term_count = term_list.count(term)
        context_prob = context_count/len(sentences)
        if context_count==0: raise ValueError

        term_context_count = 0
        for context_sentence in context_sentences:
            if len(list(re.finditer(regex_str, context_sentence.replace("\.", "."))))!=0:
                term_context_count+=1

        term_prob = term_context_count/len(sentences)
        if term_context_count==0: raise ValueError

        term_surprisal = -np.log(term_prob/context_prob)
        term_surprisals[term].append(term_surprisal)

        if term_context_count>context_count:
            print(term, context_count, term_context_count)

    with open(save_path, "wb") as f:

        pickle.dump((term_surprisals, term_types), f)
